# [실습1] Vector DB 캐싱

## 실습 목표
---
Vector DB 인덱싱 시간을 절약하기 위한 캐싱 기법을 사용해 봅니다.

## 실습 목차
---

1. **Vector DB 임베딩 캐싱:** Vector DB를 캐싱하고 저장 및 불러오는 기능을 구현합니다.

## 실습 개요
---
본격적으로 챗봇의 기능을 고도화 하기 전, 챗봇의 퀄리티를 높일 수 있는 다양한 방법을 학습합니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [7]:
import os
import time

from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import PyPDFLoader  

- Ollama를 통해 Mistral 7B 모델을 불러옵니다.

In [8]:
!ollama pull mistral:7b

'ollama'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


mistral:7b 모델을 사용하는 ChatOllama 객체를 생성합니다.

In [9]:
llm = ChatOllama(model="mistral:7b")
route_llm = ChatOllama(model="mistral:7b", format="json")
embeddings = OllamaEmbeddings(model="mistral:7b")

data_dir = "data"

## 1. Vector DB 임베딩 캐싱

4장 실습3에서 저희는 시장 조사 문건을 불러와서 `OllamaEmbeddings`를 활용해 벡터로 변환하고, FAISS DB를 활용하여 저장했습니다.
- 출처: 한국소비자원의 2022년 키오스크(무인정보단말기) 이용 실태조사 보고서
  - https://www.kca.go.kr/smartconsumer/sub.do?menukey=7301&mode=view&no=1003409523&page=2&cate=00000057

In [10]:
%%time 
# IPython의 매직 명령어 중 하나로, 셀 전체의 실행 시간을 측정합니다.

# 시장 조사 문건을 불러옵니다.
doc_path = os.path.join(data_dir, '키오스크(무인정보단말기) 이용실태 조사.pdf')
loader = PyPDFLoader(doc_path)
docs = loader.load()

vectorstore = FAISS.from_documents(
    docs,
    embedding=embeddings
)

db_retriever = vectorstore.as_retriever()

ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001FAE2E24350>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

문서 하나를 불러오는데 약 2~3분 정도 소요되었습니다.<br>
만약 사용하고자 하는 문서가 매우 많다면 챗봇을 사용하려 할 때 마다 문서를 불러오면서 많은 시간이 낭비될 것입니다.

이를 방지하기 위해, 임베딩을 마친 Vector DB를 캐싱하는 방법과, 별도로 저장하는 방법을 학습해 봅시다.

### 1.1 임베딩 캐싱

VectorStore의 `save_local` 메서드를 활용해서 임베딩 완료된 DB를 별도의 파일로 추출할 수 있으며, `load_local` 메서드를 활용해서 다시 불러올 수 있습니다.

In [5]:
#.save_local(path): 벡터 스토어를 로컬 파일 시스템에 저장하는 메서드입니다.
vectorstore.save_local("./.cache/vectorstore/키오스크(무인정보단말기) 이용실태 조사")

In [ ]:
%%time # IPython의 매직 명령어 중 하나로, 셀 전체의 실행 시간을 측정합니다.

new_vectorstore = FAISS.load_local( #.load_local(): 로컬에 저장된 벡터 스토어를 불러오는 메서드
    "./.cache/vectorstore/키오스크(무인정보단말기) 이용실태 조사",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

db_retriever = new_vectorstore.as_retriever()

불러오는 시간이 크게 단축된 것을 확인할 수 있습니다.

`load_local` 메서드를 확인하면 `allow_dangerous_deserialization` 인자가 True로 설정되어 있습니다.

FAISS DB는 로컬 파일로 저장할 때 pickle을 사용합니다. pickle 라이브러리의 보안 취약성으로 인해, Product에는 임의의 사용자가 제공한 pkl 파일을 사용하지 않는 것을 강력히 권장합니다.<br> 즉, 개발자가 서버 단에 적용한 것이 확실한 파일만 불러오거나, pickle을 사용하지 않는 ChromaDB를 사용하는 등 보안 정책을 적용해야 합니다.

## 1.2 응답 캐싱

응답을 캐싱을 사용하여, 언어 모델의 응답을 저장하여 재사용할 수도 있습니다. 
이를 통해 반복적인 질문에 대해 비용을 절약하고 응답 시간을 대폭 줄일 수 있습니다. 이렇게 하면 다음과 같은 이점이 있습니다:

- __비용 절약__: 동일한 질문에 대해 LLM을 반복 호출하지 않으므로 API 호출 비용을 절감할 수 있습니다.
- __빠른 응답__: 캐시에 저장된 결과를 즉시 반환할 수 있어, 응답 시간이 매우 빨라집니다.

LangChain에서 기본적으로 응답 캐싱(Response Caching)은 
정확히 동일한 입력에 대해서만 캐시된 응답을 반환하도록 설계되어 있습니다. 
이는 동일한 질문이 반복될 때 불필요한 API 호출을 줄이고 응답 시간을 단축하는 데 유용합니다.

캐시 메모리란 무엇인가요? 
캐시 메모리는 컴퓨터의 중앙 처리 장치(CPU)와 주 기억장치(램) 사이에 위치한 고속의 임시 저장 공간입니다. 
주 기억장치보다 훨씬 빠르게 데이터를 읽고 쓸 수 있어서, 
CPU가 자주 사용하는 데이터를 미리 저장해 두어 작업 속도를 크게 향상시킵니다.

In [ ]:
import time
from langchain.globals import set_llm_cache  # LangChain 라이브러리의 globals 모듈에서 LLM(대형 언어 모델) 캐시 설정 함수를 임포트합니다.
from langchain.cache import InMemoryCache  # LangChain 라이브러리의 cache 모듈에서 메모리 기반 캐시 클래스인 InMemoryCache를 임포트합니다.

# LLM의 응답을 캐싱하기 위해 InMemoryCache의 인스턴스를 생성하고 설정합니다.
# 캐시를 설정함으로써 동일한 입력에 대해 반복적으로 LLM을 호출하지 않고, 캐시된 결과를 재사용하여 성능을 향상시킬 수 있습니다.
# 이는 특히 LLM 호출이 비용이 많이 들거나 시간이 오래 걸리는 경우 유용합니다.
set_llm_cache(InMemoryCache())

start_time = time.time()

response = llm.invoke("한국의 수도에 대해서 설명해줘.")

time_passed = time.time() - start_time

print(f"답변: {response.content}, 소요 시간: {round(time_passed, 2)} 초")

InMemoryCache()

메모리(컴퓨터의 주기억장치)에 데이터를 저장하는 캐시를 만듭니다.
저장된 데이터는 프로그램이 실행되는 동안만 유지됩니다.

set_llm_cache(...)

위에서 만든 메모리 캐시를 LangChain 라이브러리에 설정합니다.
이렇게 설정하면 LangChain이 LLM에게 요청할 때마다 이 캐시를 먼저 확인하게 됩니다.

In [ ]:
start_time = time.time()

response = llm.invoke("한국의 수도에 대해서 설명해줘.")

time_passed = time.time() - start_time

print(f"답변: {response.content}, 소요 시간: {round(time_passed, 2)} 초")